<a href="https://colab.research.google.com/github/Meaveryway/Backend_AntTech/blob/master/Extraction_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install pdfplumber
import pdfplumber
!pip install tika
import tika
tika.initVM()

  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32893 sha256=43490c393f6f9e9cdc82bff85aea01e06a8bc5dda0ebc197c337e20bb760f8ae
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika


####Schema
Then differentiate text fields for arabic and french

In [ ]:
code = {
    'id':
    'category':
    'book_num':
    'book':
    'title_num':
    'title':
    'chapter_num'
    'chapter':
    'article_num':
    'content':
    'status':

    'extraction_date':
    
}

####File parsing
1. Split pdf by page
2. Convert to word (for Ar)
3. Extract

In [5]:
CODE_CIVIL_AR_PATH = "/content/ACivil.pdf"
CODE_CIVIL_AR_PAGE_RANGE = [8, 171]

In [163]:
import pdfplumber
def digitize_file_pdfplumber(file_path):
  text = ""
  with pdfplumber.open(file_path) as pdf:
    page = pdf.pages[0].dedupe_chars(tolerance=1)
    text = text + page.extract_text()
  pdf.close()
  return(text)
  
def roman_to_int(s):
  rom_val = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
  int_val = 0
  for i in range(len(s)):
    if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
      int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
    else:
      int_val += rom_val[s[i]]
  return int_val

In [309]:
extract = digitize_file_pdfplumber("/content/samp/document-page19.pdf")
import re
_RE_COMBINE_WHITESPACE = re.compile(r"\s+")

[INFO] xref found: pos=b'81503'
[INFO] read_xref_from: start=81503, token=/b'xref'
[INFO] xref objects: {1: (None, 9, 0), 2: (None, 68, 0), 3: (None, 108, 0), 4: (None, 263, 0), 5: (None, 312, 0), 6: (None, 1177, 0), 7: (None, 1226, 0), 8: (None, 1796, 0), 9: (None, 1905, 0), 10: (None, 2032, 0), 11: (None, 2182, 0), 12: (None, 2866, 0), 13: (None, 3066, 0), 14: (None, 41358, 0), 15: (None, 41820, 0), 16: (None, 41975, 0), 17: (None, 42645, 0), 18: (None, 42852, 0), 19: (None, 75675, 0), 20: (None, 76150, 0)}
[INFO] trailer: {'Size': 21, 'Root': <PDFObjRef:4>, 'Info': <PDFObjRef:2>}
[INFO] trailer: {'Size': 21, 'Root': <PDFObjRef:4>, 'Info': <PDFObjRef:2>}
[INFO] Pages: Kids=[<PDFObjRef:3>]
[INFO] Page: {'CropBox': [0, 0, 595, 842], 'Rotate': 0, 'Parent': <PDFObjRef:1>, 'Thumb': <PDFObjRef:5>, 'Resources': <PDFObjRef:8>, 'MediaBox': [0, 0, 595, 842], 'Type': /'Page', 'Contents': <PDFObjRef:20>}
[INFO] Processing page: <PDFPage: Resources={'ExtGState': {'GS1': <PDFObjRef:9>}, 'Font': {'

In [333]:
test = "1 -  of shit. this is what i don't need"
if re.match(r'\d\ -.*\.', test):
  print('ok')

ok


In [341]:
text = _RE_COMBINE_WHITESPACE.sub(" ", extract).strip()
while len(text) > 0 and 'Art' in text:
  if 'LIVRE' in text[:20]:
    print('*found book')
    book = text.split('LIVRE')[-1].split('TITRE')[0].strip()

    if len(book) > 150 or len(book) < 15:
      raise Exception(f"Failed to extract book element: /{book}/")    
    text = text[text.index(book) + len(book):].strip() #purging mother text

    book_num = book.split(' ')[0].strip()
    book_num = roman_to_int(book_num)
    book_label = " ".join(book.split(' ')[1:])
    print(f"/{book}/, /{book_num}/, /{book_label}/")

  elif 'TITRE' in text[:20]:

    print('*found title')

    title_chapter = text.split('TITRE')[-1].split('Chapitre')[0].strip() #trying to split with the presence of 'Chapter ...' after the title
    title_article = text.split('TITRE')[-1].split('Art')[0].strip() #trying to split with the presence of 'Art.' or 'Article 1er.' after the title
    title = min([title_chapter, title_article], key=len)
    if len(title) > 150 or len(title) < 15:
      raise Exception(f"Failed to extract title element: /{title}/")
    text = text[text.index(title) + len(title):].strip() #purging mother text

    title_num = title.split(' ')[0].strip()
    title_num = roman_to_int(title_num)
    title_label = " ".join(book.split(' ')[1:])
    print(f"/{title}/, /{title_num}/, /{title_label}/")

  elif 'Chapitre' in text[:20]:
    print('*found chapter')
    chapter_section = text.split('Chapitre')[1].split('Section')[0].strip() #trying to split with the presence of 'Section ...' after the title
    chapter_article = text.split('Chapitre')[1].split('Art')[0].strip() #trying to split with the presence of 'Art.' or 'Article 1er.' after the title
    chapter = min([chapter_section, chapter_article], key=len)
    
    if len(chapter) > 150 or len(chapter) < 10:
      raise Exception(f"Failed to extract chapter element: /{chapter}/")
    text = text[text.index(chapter) + len(chapter):].strip() #purging mother text
    
    chapter_num = chapter.split(' ')[0].strip()
    chapter_num = roman_to_int(chapter_num)
    chapter_label = " ".join(chapter.split(' ')[1:])
    print(f"/{chapter}/, /{chapter_num}/, /{chapter_label}/")

  elif 'Section' in text[:20]:
    print('*found Section')
    section_article = text.split('Section')[-1].split('Art')[0].strip() #trying to split with the presence of 'Art.' or 'Article 1er.' after the section title
    section_subsection = re.split(r'\d\ -.*\.', text.split('Section')[-1])[0].strip() #trying to split with the presence of a subsection '1 - something' after the section title
    
    section = min([section_article, section_subsection], key=len)
    
    if len(section) > 150 or len(section) < 10:
      raise Exception(f"Failed to extract section element: /{section}/")
    text = text[text.index(section) + len(section):].strip() #purging mother text
    
    section_num = section.split(' ')[0].strip()
    section_num = roman_to_int(section_num)
    section_label = " ".join(section.split(' ')[1:])
    print(f"/{section}/, /{section_num}/, /{section_label}/")

  #subsection
  elif re.match(r'\d\ -.*\.', text[:20]):
    print('*found Sub-section')
    subsection = text.split('Art')[0].strip() #trying to split with the presence of 'Art.' or 'Article 1er.' after the sub-section title    
    
    if len(subsection) > 150 or len(subsection) < 5:
      raise Exception(f"Failed to extract sub-section element: /{subsection}/")
    text = text[text.index(subsection) + len(subsection):].strip() #purging mother text
    
    subsection_num = subsection.split('-')[0].strip()
    subsection_num = re.sub(r"\D", "", subsection_num)

    subsection_label = subsection[subsection.index('-') + 1 : subsection.rfind('.')].strip()
    print(f"/{subsection}/, /{subsection_num}/, /{subsection_label}/")

  elif 'Art.' in text[:10]:
    print('*found article')
    article_article = text.split('Art.')[1]
    article_chapter = text.split('Art.')[1].split('Chapitre')[0]
    article_section = text.split('Art.')[1].split('Section')[0]
    article_end = text.split('Art.')[1].split('_____')[0] #to capture last article in page
    article = min([article_article, article_chapter, article_section, article_end], key=len)


    if len(article) < 20:
      raise Exception(f"Failed to extract article element: /{article}/")
    text = text[text.index(article) + len(article):].strip() #purging mother text

    identifier = article.split('-')[0]
    article_identifier = identifier.split('.')[0]
    article_num = re.sub(r"\D", "", article_identifier)
    article_subnum = article_identifier.replace(article_num, ' ').strip()
    article_status = identifier[identifier.rfind('('): identifier.rfind(')')]
    article_status = re.sub("[^A-Za-zé]", "", article_status)

    article_text = article[article.index('-') + 1 : article.rfind('.') + 1].strip() #rfind to deal with cases where there's no footer ___ on the page
    note_num = article[article.rfind('.') + 1:].strip()
    note_num = re.sub(r"\D", "", note_num)
    print(f"#{article_num}, {article_subnum} ({article_status}) [note: {note_num}]: {article_text}")
#print(f"----- {text}")
#print(len(articles))

*found Section
/II DES CONDITIONS DU CONTRAT/, /2/, /DES CONDITIONS DU CONTRAT/
*found Sub-section
/1 - Du consentement./, /1/, /Du consentement/
*found article
#59,  () [note: ]: Le contrat se forme dès que les parties ont échangé leurs volontés concordantes, sans préjudice des dispositions légales.
*found article
#60,  () [note: ]: On peut déclarer sa volonté verbalement, par écrit ou par les signes généralement en usage ou encore par une conduite telle qu'elle ne laisse aucun doute sur la véritable intention de son auteur. La déclaration de volonté peut être tacite lorsque la loi ou les parties n'exigent pas qu'elle soit expresse.
*found article
#61,  () [note: ]: Une déclaration de volonté produit son effet dès qu'elle parvient à la connaissance de son destinataire. Celui-ci sera réputé avoir pris connaissance de la déclaration dès sa réception, à moins de preuve contraire.
*found article
#62,  () [note: ]: Si l'auteur de la déclaration décède ou devient incapable avant que celle-c

In [149]:
for article in text.split('Art.')[1:]:
  try:
    article_num = article.split('.', 1)[0].strip()
    contains_number = re.sub(r"\D", "", article_num)
    if not contains_number.isnumeric():
      raise Exception("Not an article")
    print(f"-{article_num}-")
  except:
    print(f"Not an article: /{article}/")

-2-
-3-
-4-
-5-


####Tika

In [315]:
data = digitize_file_tika("/content/samp/document-page19.pdf")

In [316]:
data

'<html xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<meta name="pdf:PDFVersion" content="1.3" />\n<meta name="X-Parsed-By" content="org.apache.tika.parser.DefaultParser" />\n<meta name="X-Parsed-By" content="org.apache.tika.parser.pdf.PDFParser" />\n<meta name="pdf:hasXFA" content="false" />\n<meta name="access_permission:modify_annotations" content="true" />\n<meta name="access_permission:can_print_degraded" content="true" />\n<meta name="access_permission:extract_for_accessibility" content="true" />\n<meta name="access_permission:assemble_document" content="true" />\n<meta name="xmpTPg:NPages" content="1" />\n<meta name="resourceName" content="b\'document-page19.pdf\'" />\n<meta name="pdf:hasXMP" content="false" />\n<meta name="dc:format" content="application/pdf; version=1.3" />\n<meta name="access_permission:extract_content" content="true" />\n<meta name="access_permission:can_print" content="true" />\n<meta name="access_permission:fill_in_form" content="true" />\n<meta name="pdf

In [ ]:
xhtml_data = BeautifulSoup(data)
for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
    print('Parsing page {} of pdf file...'.format(page+1))
    print(content)
    #_buffer.write(str(content))
    #parsed_content = parser.from_buffer(_buffer.getvalue())
    #_buffer.truncate()
    #file_data.append({'id': 'page_'+str(page+1), 'content': parsed_content['content']})

In [314]:
from tika import parser
from bs4 import BeautifulSoup
from io import StringIO
from tika import detector
def digitize_file_tika(file_path):
    file_extension = detector.from_file(file_path)
    if False:#'pdf'in str(file_extension).lower():
      file_data = []
      buffer = StringIO()
      data = parser.from_file(file_path, xmlContent=True)
      xhtml_data = BeautifulSoup(data['content'], features="lxml")
      for page, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
          #print('Parsing page {} of pdf file...'.format(page+1))
          buffer.write(str(content))
          parsed_content = parser.from_buffer(buffer.getvalue())
          buffer.truncate()
          file_data.append({'id': str(page+1), 'content': parsed_content['content'].replace('\n\n', '').replace('\n', ' ')})
      #Clean-up
      for i in reversed(range(1,len(file_data))) :
        content = file_data[i]['content']
        substract = file_data[i-1]['content']
        modified_content = content.replace(substract, "")
        file_data[i]['content'] = modified_content
      return file_data
    
    else:
      text = data = parser.from_file(file_path, xmlContent=True)['content']
      return(text)

    

####Split

In [56]:
!pip install pypdf2

     |████████████████████████████████| 77 kB 4.0 MB/s 
  Created wheel for pypdf2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61100 sha256=6a328fed4ac07955ba1d7ef479ddcc615c80652b2d9a4895d43270d00d9ef560
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built pypdf2


In [81]:
from PyPDF2 import PdfFileWriter, PdfFileReader

inputpdf = PdfFileReader(open("/content/FCivil.pdf", "rb"))

for i in range(inputpdf.numPages):
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open("/content/samp/document-page%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)

####Convert to Docx

In [ ]:
!pip install pdf2docx
from pdf2docx import Converter
import os

In [59]:
cv = Converter('/content/document-page0.pdf')
cv.convert('/content/ACivilsamp.docx', start=0, end=None)
cv.close()

[INFO] Start to convert /content/document-page0.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[INFO] [3/4] Parsing pages...
[INFO] (1/1) Page 1
[INFO] [4/4] Creating pages...
[INFO] (1/1) Page 1
[INFO] Terminated in 0.79s.


In [ ]:
# # # dir_path for input reading and output files & a for loop # # #

path_input = '/pdftodocx/input/'
path_output = '/pdftodocx/output/'

for file in os.listdir(path_input):
    cv = Converter(path_input+file)
    cv.convert(path_output+file+'.docx', start=0, end=None)
    cv.close()
    print(file)